In [32]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

from pathlib import Path
import os
import sys
from datetime import datetime, date

from random import sample
import random

import time


In [33]:
result = Path('result/')
report = Path('report/')
instances = Path('instances/csifa')

In [34]:
file_name = sys.argv[1]
#tam_part = int(sys.argv[2]) # Tamanho das partições
#num_fix = int(sys.argv[3]) # número de variáveis que serão fixadas 

In [35]:
def read_instance(file_name):
	
	arq = open(file_name)
	
	N = int(arq.readline())
	
	PR  = [0]*N
	PP  = [0]*N

	FR = [float(arq.readline())]*N
	FP = [float(arq.readline())]*N

	HR = [float(arq.readline())]*N
	HP = [float(arq.readline())]*N

	D = [int(i) for i in arq.readline().split()]
	
	R = [int(i) for i in arq.readline().split()]

	C = float(arq.readline().rstrip('\n'))
	
	return N, PP, PR, FP, FR, HR, HP, D, R, C

In [36]:
def run_time(start_time=None):

	if not start_time:
		start_time = datetime.now()
		return start_time
	
	elif start_time:
		temp_sec = (datetime.now() - start_time).total_seconds()
		return temp_sec

In [37]:
def gera_particoes(N, tam_part=5, num_part_fix=2, seme=5, ind_gen=1):
	
	tam_jan = tam_part - num_part_fix
	subset = []
	 
	if ind_gen == 1:
		for i in range(0, N, tam_jan):
			if i + tam_part > N:
				subset.append([k for k in range(i,N)])
			else:
				subset.append([k for k in range(i,i+tam_part)])
	else:
		random.seed(seme)
		list_per = [k  for k in range(N)]
	
		while True:		 
			sub_conj = []
			 
			if len(list_per) >= tam_part:
				sub_conj = sample(list_per, tam_part)
			else:
				sub_conj = list_per[:]
			subset.append(sub_conj)

			for i in sub_conj:
				list_per.remove(i)
			
			if len(list_per)==0:
				break
	
	return subset

In [38]:
MAX_CPU_TIME = 3600.0
EPSILON = 1.e-6

def relax_and_fix(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C, parts, yp_sol, yr_sol):

	objval = 0.0

	try:

		# Create a new model
		model = gp.Model("CLSR")

		# Create variables
		xp = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xp")
		yp = model.addVars(list(range(N)), vtype=GRB.BINARY, name="yp")
		sp = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="sp")
		xr = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xr")
		yr = model.addVars(list(range(N)), vtype=GRB.BINARY, name="yr")
		sr = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="sr")
		
		for i in range(N):
			if i > max(parts) :
				yp[i].VType = gp.GRB.CONTINUOUS
				yp[i].lb    = 0.0
				yp[i].ub    = 1.0
				yr[i].VType = gp.GRB.CONTINUOUS
				yr[i].lb    = 0.0
				yr[i].ub    = 1.0
			elif i in parts:
				yp[i].lb = 0.0
				yp[i].ub = 1.0
				yr[i].lb = 0.0
				yr[i].ub = 1.0
			else:
				yp[i].lb = yp_sol[i]
				yp[i].ub = yp_sol[i]
				yr[i].lb = yr_sol[i]
				yr[i].ub = yr_sol[i]
		
		model.update()

		# Set objective
		model.setObjective(gp.quicksum(PP[i]*xp[i]+sp[i]*HP[i] + xr[i]*PR[i] + sr[i]*HR[i] + yp[i]*FP[i] + yr[i]*FR[i] for i in range(N)), sense=GRB.MINIMIZE)

		# Add constraints
		model.addConstr(xp[0] + xr[0] - sp[0] == D[0])

		model.addConstrs(sp[i-1] + xp[i] + xr[i] - sp[i] == D[i] for i in range(N) if i > 0 )
		
		model.addConstr(R[0] - xr[0] - sr[0] == 0)
		
		model.addConstrs(sr[i-1] + R[i] - xr[i] - sr[i] == 0 for i in range(N) if i > 0)
		
		model.addConstrs(xp[i] - yp[i]*min(C,SD[i][N-1]) <= 0 for i in range(N))
		
		model.addConstrs(xr[i] - yr[i]*min(SR[0][i], SD[i][N-1]) <= 0 for i in range(N))
		
		model.addConstrs(xp[i] + xr[i] <= C for i in range(N))
		
		# Parameters 
		model.setParam(GRB.Param.TimeLimit, MAX_CPU_TIME)
		model.setParam(GRB.Param.MIPGap, EPSILON)
		model.setParam(GRB.Param.Threads,1)
		#model.setParam(GRB.Param.Cuts,-1)
		#model.setParam(GRB.Param.Presolve,-1)

		# Optimize model
		model.optimize()

		objval = model.ObjVal

		yp_val = [0]*N
		yr_val = [0]*N

		yp_val = [yp[i].X for i in range(N)]
		yr_val = [yr[i].X for i in range(N)]

	except gp.GurobiError as e:
		print('Error code ' + str(e.errno) + ': ' + str(e))

	#except AttributeError:
	#	print('Encountered an attribute error')

	return objval, yp_val, yr_val

In [41]:
def main(file_name):

	solution = 0.0

	tam_part = 5
	num_fix = 2
	
	N, PP, PR, FP, FR, HR, HP, D, R, C = read_instance(os.path.join(instances,file_name))

	SD = (np.zeros((N,N))).tolist()
	SR = (np.zeros((N,N))).tolist()
	for  i in range(N):
		SD[i][i] = D[i]
		SR[i][i] = R[i]
		for j in range(i+1,N):
			SD[i][j] = SD[i][j-1] + D[j]
			SR[i][j] = SR[i][j-1] + R[j]

	subset = gera_particoes(N, tam_part, num_fix)

	yp_sol = [0]*N
	yr_sol = [0]*N

	start_time = time.time()
	
	for s in subset:
		solution, yp_sol, yr_sol = relax_and_fix(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C, s, yp_sol, yr_sol)
		 
	run_time = time.time() - start_time
	
	arquivo = open(os.path.join(result,'clsr_std_rf.txt'),'a')
	arquivo.write(file_name+';'+str(round(solution,2))+';'+str(round(run_time,2))+'\n')

	arquivo.close()

In [42]:
if __name__== "__main__" :

	file_name = f"c52_1.txt"

	main(file_name)

Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 1e-06
Set parameter Threads to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads

Optimize a model with 260 rows, 312 columns and 674 nonzeros
Model fingerprint: 0xddc53cb8
Variable types: 302 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e-01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+02]
Presolve removed 99 rows and 99 columns
Presolve time: 0.00s
Presolved: 161 rows, 213 columns, 477 nonzeros
Variable types: 204 continuous, 9 integer (9 binary)
Found heuristic solution: objective 6579.1075856

Root relaxation: objective 5.376769e+03, 150 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds 

     0     0 5532.81539    0    4 5668.43592 5532.81539  2.39%     -    0s
H    0     0                    5572.0123475 5532.81539  0.70%     -    0s
     0     0 5560.21336    0    4 5572.01235 5560.21336  0.21%     -    0s
     0     0     cutoff    0      5572.01235 5572.01235  0.00%     -    0s

Cutting planes:
  Gomory: 1
  Cover: 1
  Implied bound: 3
  MIR: 1
  Flow cover: 8

Explored 1 nodes (164 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 4: 5572.01 5668.44 5698.88 6579.11 

Optimal solution found (tolerance 1.00e-06)
Best objective 5.572012347534e+03, best bound 5.572012347534e+03, gap 0.0000%
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 1e-06
Set parameter Threads to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, usi